### 학습 데이터 범위
* 1안 [현안]  
: {양호 , 불량부분(파손된 부분)} 학습

~~*2안~~  
~~:{양호, 불량전체(파손된 부분을 포함하는 객체 전체를 의미하는 듯함), 불량부분} 학습~~

~~*3안~~  
~~:{불량부분} 학습~~


### label

배경 0 

점자블럭 1 (파손부)

보도블럭 2 (파손부)

자전거도로 3 (파손부)

### format 변환

한 객체에 대한 Bound 를 나타내는 다각형의 점들의 집합  
**P = [(x1, y1), (x2, y2), ... (xn, yn)]**
**P_X = [x1, x2, ... , xn]**
**P_Y = [y1, y2, ... , yn]**

P -> P' 으로 사이즈 변환 및 좌표값 Normalizing 매핑을 위해서는,  

**(사이즈 변환에 의한 좌표 조정값) X (Normalizing Term)**  

x_ratio = (target_width / origin_width) * (1 / target_width) = 1 / origin_width  

y_ratio = (target_height / origin_height) * (1 / target_height) = 1 / origin_height  

P' = [(x1', y1'), (x2', y2'), ... (xn', yn')]

P_X' = [x1' , x2', ... xn'] = x_ratio x P_X

P_Y' = [y1', y2', ...yn'] = y_ratio x P_Y


x_min = minimum x coordinate in P'

x_max = maximum x coordinate in P'

y_min = minimum y coordinate in P'

y_max = maximum y coordinate in P'

최종적으로 다음의 포맷으로 변환하기 위한 관계식은

**class_id, center_x, center_y, width, height**

width = (x_max - x_min)

height = (y_max - y_min)

center_x = (x_min + x_max) / 2

center_y = (y_min + y_max) / 2

### 함수 정의

In [7]:
import json
import numpy as np
# from PIL import Image

# 단순히 사용 데이터셋의 .json 파일에서 필요한 정보를 추출하는 함수
def get_bounding_boxes_from(data : json):
    bounding_box_list = []
    labels = {
              "점자블럭" : 0,
              "보도블록" : 1,
              "자전거 도로" : 2,
              }
    # 분류할 타입
    # is_defect = ["정상", "불량부분"]
    is_defect = ["불량부분"]
    
    # labels
    for object in data["annotations"]:
        # is_defect 내에 포함되지 않으면 bounding box 를 생성하지 않음
        if not object["is_defect"] in is_defect: continue

        ## label info parsing ##

        encoded_label = 0 # default value
        try:
            label_name = object["label_name"]
            encoded_label = labels[label_name]
        except :
            print(f"잘못된 label_name : {label_name}")
            return
        
        ## bounding box parsing ##

        x_min = y_min = 9999
        x_max = y_max = 0
        for vertex in object["annotation_info"]:
            x = vertex[0]
            y = vertex[1]
            x_max = max(x, x_max)
            x_min = min(x, x_min)
            y_max = max(y, y_max)
            y_min = min(y, y_min)

        bounding_box_list.append([encoded_label, x_min, x_max, y_min, y_max])
        # print(bounding_box_list)
    
    bounding_boxes = np.array(bounding_box_list)
    
    return bounding_boxes


# YOLO 모델의 .txt 라벨데이터 포맷에 맞게 Bounding box 정보 변환
# 정사각 이미지로 만들기 위해 padding 을 덧댄 경우
def trans_format_with_padding(bounding_boxes : np.ndarray, json_file : json):
    N_digits = 8
    # class_id center_x center_y width height  
    # 원본 이미지 사이즈
    width, height = json_file["info"]["width"], json_file["info"]["height"]

    long_edge = max(width, height)
    short_edge = min(width, height)
    
    padding = (long_edge - short_edge)/2
    
    target_size = 640.0 # 축소할 사이즈
    origin_size = long_edge # 패딩을 추가한 이미지는 long_edge X long_edge 의 사이즈가 된다.

    # where fields are space delimited, and the coordinates are normalized from zero to one.
    # Note: To convert to normalized xywh from pixel values, 
    # divide x (and width) by the image's width and divide y (and height) by the image's height.
    
    # (사이즈 변환에 의한 좌표 조정값) * (Normalizing Term)
    # (target_size / origin_size) * (1 / target_size) = 1 / origin_size
    ratio = round(1 / origin_size, ndigits=N_digits)

    objects = []
    for box in bounding_boxes:
        label = box[0]
    
        # width < heigth : 패딩이 x 축에만 추가
        # width > height : 패딩은 y 축에만 추가
        if (width < height) : 
            x_min = ratio * (box[1] + padding) # x_min
            x_max = ratio * (box[2] + padding) # x_max
            y_min = ratio * box[3] # y_min
            y_max = ratio * box[4] # y_max
        else:
            x_min = ratio * box[1] # x_min
            x_max = ratio * box[2] # x_max
            y_min = ratio * (box[3] + padding) # y_min
            y_max = ratio * (box[4] + padding)# y_max

    
        center_x = round((x_min + x_max)/2, ndigits=N_digits)
        center_y = round((y_min + y_max)/2, ndigits=N_digits)
        width = x_max - x_min
        height = y_max - y_min
        line = np.array([label, center_x, center_y, width, height])
        objects.append(line)

    return np.array(objects)

# 패딩을 덧대지 않은 경우
# 정사각형 사이즈 이미지가 아니기 때문에 좌표 변환 과정이 살짝 다르다.
def trans_format_without_padding(bounding_boxes : np.ndarray, json_file : json):
    N_digits = 8
    # class_id center_x center_y width height  
    # 원본 이미지 사이즈
    origin_width, origin_height = json_file["info"]["width"], json_file["info"]["height"]
    
    # where fields are space delimited, and the coordinates are normalized from zero to one.
    # Note: To convert to normalized xywh from pixel values, 
    # divide x (and width) by the image's width and divide y (and height) by the image's height.
    
    # (사이즈 변환에 의한 좌표 조정값) * (Normalizing Term)
    # (target_width / origin_width) * (1 / target_width) = 1 / origin_width
    x_ratio = round(1 / origin_width, ndigits=N_digits)
    y_ratio = round(1 / origin_height, ndigits=N_digits)

    objects = []
    for box in bounding_boxes:
        label = box[0]

        x_min = x_ratio * box[1] # x_min
        x_max = x_ratio * box[2] # x_max
        y_min = y_ratio * box[3] # y_min
        y_max = y_ratio * box[4] # y_max
            
        center_x = round((x_min + x_max)/2, ndigits=N_digits)
        center_y = round((y_min + y_max)/2, ndigits=N_digits)
        width = x_max - x_min
        height = y_max - y_min
        line = np.array([label, center_x, center_y, width, height])
        objects.append(line)

    return np.array(objects)

테스트 코드

In [ ]:
### test code ###

# apple
# 2_09_1_1_1_1_20210719_0000006002.jpg
# samsung
# 2_09_1_1_1_1_20210716_0000037039.jpg


root_path = "D:/Downloads/street-facilities-selected"

# empty list [] 반환
# file = "2_09_0_1_4_1_20210927_0000569294" # 정상

file = "2_09_1_1_1_1_20210904_0000469427" # 점자 블록 파손

image_path = f"{root_path}/images/{file}.jpg" # .jpg or .jpeg
json_path = f"{root_path}/labels/{file}.json"

with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

bounding_boxes = get_bounding_boxes_from(data)

print(image_path)

print("bounding boxes")
for box in bounding_boxes:
    print(box)

# translated_format = trans_format_with_padding(bounding_boxes, data)
translated_format = trans_format_without_padding(bounding_boxes, data)
print("\ntranslated bounding boxes")
for t in translated_format:
    print(t)

np.savetxt(f"./{file}.txt", translated_format, delimiter=" ", fmt='%.8f')

D:/Downloads/street-facilities-selected/images/2_09_1_1_1_1_20210904_0000469427.jpg
bounding boxes
[1.00000000e+00 7.87310498e+02 1.32771918e+03 1.60915200e+03
 2.16869486e+03]
[1.00000000e+00 1.16119456e+03 1.23122818e+03 1.29422213e+03
 1.37276758e+03]

translated bounding boxes
[1.         0.46627944 0.46849079 0.238277   0.13877782]
[1.         0.52743352 0.33073339 0.03087922 0.01948084]


### 변환

In [8]:
import os
import json
import numpy as np

root_path = "D:/Downloads/street-facilities-selected"
json_path = f"{root_path}/labels"
save_path = "D:/Downloads/street-facilities-selected/labels-txt"

json_files = [f for f in os.listdir(json_path) if f.endswith('.json')]

total = len(json_files)

print(json_path)

# 폴더에 있는 .json 파일 순회
for i, filename in enumerate(json_files):
    filename = os.path.splitext(filename)[0] # .json 확장자명 제거
    if(i%300==0):
        print(f"{i+1} / {total}")

    with open(f"{json_path}/{filename}.json", "r", encoding="utf-8") as f:
        data = json.load(f)
        bounding_boxes = get_bounding_boxes_from(data)
        # translated_format = trans_format_with_padding(bounding_boxes, data)
        translated_format = trans_format_without_padding(bounding_boxes, data)
        np.savetxt(f"{save_path}/{filename}.txt", translated_format, delimiter=" ", fmt='%.8f')

D:/Downloads/street-facilities-selected/labels
1 / 8477
301 / 8477
601 / 8477
901 / 8477
1201 / 8477
1501 / 8477
1801 / 8477
2101 / 8477
2401 / 8477
2701 / 8477
3001 / 8477
3301 / 8477
3601 / 8477
3901 / 8477
4201 / 8477
4501 / 8477
4801 / 8477
5101 / 8477
5401 / 8477
5701 / 8477
6001 / 8477
6301 / 8477
6601 / 8477
6901 / 8477
7201 / 8477
7501 / 8477
7801 / 8477
8101 / 8477
8401 / 8477


classId 는 Int 형이여야 하므로 이에 대한 처리 수행

In [9]:
import os

root_path = "D:/Downloads/street-facilities-selected"

label_path = f"{root_path}/labels-txt"

labels = [label for label in os.listdir(label_path) if label.endswith('.txt')]

for label in labels:
    annotations = []
    with open(os.path.join(label_path, label), "r+", encoding="utf-8") as f:
        # 기존 내용 읽기 및 수정
        for line in f:
            elems = line.split(" ")
            elems[0] = str(int(float(elems[0]))) # 1.00000000 -> 1 -> '1'
            newline = " ".join(elems)
            annotations.append(newline)
            
        # 새 내용 작성
        f.seek(0) # 파일 포인터 이동
        for annotation in annotations:
            f.write(annotation)
        f.truncate() # 파일 포인터 이전 내용만 남기고 나머지 삭제

누락된 파일 없는지 갯수 체크

In [ ]:
import os

root_path = "D:/Downloads/street-facilities-selected"
json_path = f"{root_path}/labels"
save_path = "D:/Downloads/street-facilities-selected/labels-txt"

json_files = [f for f in os.listdir(json_path) if f.endswith('.json')]
json_total = len(json_files)
print("original json file counts : ", json_total)

txt_files = [f for f in os.listdir(save_path) if f.endswith('.txt')]
txt_total = len(txt_files)
print("txt file counts : ", txt_total)